In [1]:
import numpy as np
import pandas as pd 

import torch
import tensorflow as tf 
from tensorflow import keras 

from sklearn.model_selection import train_test_split

2023-11-05 02:19:32.969430: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
data =  pd.read_csv(
    '/Users/Labi/V_env/deploy-tf-torch/data/output/train/train.csv', 
    names=[
    'months_as_member', 
    'weight', 
    'days_before', 
    'day_of_week', 
    'time', 
    'attended'
]
)

In [11]:
data.head()

,months_as_member,weight,days_before,day_of_week,time,attended
0,10,85.31,12,6,2,0
1,10,88.58,2,1,1,1
2,6,90.60,14,6,2,0
3,11,69.61,10,5,1,1
4,33,81.68,8,4,1,1


In [42]:
val_dataframe = data.sample(frac=0.2, random_state=1337)
train_dataframe = data.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 947 samples for training and 237 for validation


##### TENSORFLOW

##### TESTING FUNCTIONAL API

In [14]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("attended")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [16]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print('\n')
    print("Target:", y)

Input: {'months_as_member': <tf.Tensor: shape=(), dtype=int64, numpy=6>, 'weight': <tf.Tensor: shape=(), dtype=float64, numpy=111.16>, 'days_before': <tf.Tensor: shape=(), dtype=int64, numpy=10>, 'day_of_week': <tf.Tensor: shape=(), dtype=int64, numpy=5>, 'time': <tf.Tensor: shape=(), dtype=int64, numpy=1>}


Target: tf.Tensor(0, shape=(), dtype=int64)


In [17]:
train_ds = train_ds.batch(16)
val_ds = val_ds.batch(16)

In [27]:
train_ds

<_BatchDataset element_spec=({'months_as_member': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'weight': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'days_before': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'day_of_week': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'time': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [37]:
months_as_member = keras.Input(shape=(1,), name="months_as_member", dtype="int64")
# weight = keras.Input(shape=(1,), name="weight", dtype="float64")
days_before = keras.Input(shape=(1,), name="days_before", dtype="int64")
day_of_week = keras.Input(shape=(1,), name="day_of_week", dtype="int64")
time = keras.Input(shape=(1,), name="time", dtype="int64")


##

In [48]:
model = keras.Sequential([
    keras.layers.Dense(32, activation="relu", input_shape=(train_dataframe.shape[1]-1,)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(
    train_dataframe.iloc[:, :-1], 
    train_dataframe.iloc[:, -1], 
    epochs=50,
    validation_data=(
        val_dataframe.iloc[:, :-1], 
        val_dataframe.iloc[:, -1]
    )
)

In [50]:
model.save('./models/tf_model', save_format='tf')

INFO:tensorflow:Assets written to: ./models/tf_model/assets


INFO:tensorflow:Assets written to: ./models/tf_model/assets


##### TORCH

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [6]:
class SimpleModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out


In [43]:
input_size = data.shape[1] -1
hidden_size = 32
output_size = 1 

model = SimpleModel(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Print the model architecture
model

SimpleModel(
  (fc1): Linear(in_features=5, out_features=32, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [5]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1],data.iloc[:, -1], test_size=0.2)

In [44]:
class TrainSet(Dataset):
    def __init__(self, X, y):
        self.n_samples = len(X)
        self.x = X
        self.y = y

    def __getitem__(self, index: int)-> tuple:
        x_data = torch.tensor(self.x.values, dtype=torch.float32)
        y_data = torch.tensor(self.y.values, dtype=torch.float32)
      
        return x_data[index], y_data[index]

    def __len__(self) -> int:
        return self.n_samples

In [28]:
type(X_train)

pandas.core.frame.DataFrame

In [29]:
train_set = TrainSet(
    X_train, 
    y_train
)
loader = DataLoader(dataset = train_set, batch_size = 8, shuffle = True, num_workers = 0)                

In [45]:
epochs = 50
for epoch in range(epochs):
    for (feat, target) in loader:
        optimizer.zero_grad()
        outputs = model(feat)
        loss = criterion(outputs, target.view(-1, 1)) 
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"epoch {epoch+1}/{epochs}, loss={loss.item():.4f}")

epoch 10/50, loss=0.1405
epoch 20/50, loss=0.1851
epoch 30/50, loss=0.2236
epoch 40/50, loss=0.3907
epoch 50/50, loss=0.4150


In [49]:
torch.save(model, './models/torch_model.pth')

In [33]:
import tensorflow as tf

In [71]:
tf_mod  = tf.keras.models.load_model('models/tf_model')

In [51]:
tor_mod = torch.load('models/torch_model.pth')

In [78]:
X_test.iloc[0, :].values.tolist()

[14.0, 76.24, 10.0, 5.0, 1.0]

In [82]:

(tf.argmax(tf_mod.predict(X_test.iloc[:2, :].values.tolist()), axis=1).numpy()).tolist()

1/1 [==============================] - 0s 258ms/step


[0, 0]

In [81]:
np.where(tor_mod(torch.tensor(X_test.iloc[:2, :].values.tolist(), dtype=torch.float32)).detach().numpy()[:,0]<0.5 , 0, 1).tolist()

[0, 1]